## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
import time
import numpy as np

In [3]:
import logging
import contextlib
from http.client import HTTPConnection

def debug_requests_on():
    '''Switches on logging of the requests module.'''
    HTTPConnection.debuglevel = 1

    logging.basicConfig(filename = 'requestslog.log')
    logging.getLogger().setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

def debug_requests_off():
    '''Switches off logging of the requests module, might be some side-effects'''
    HTTPConnection.debuglevel = 0

    root_logger = logging.getLogger()
    root_logger.setLevel(logging.WARNING)
    root_logger.handlers = []
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.WARNING)
    requests_log.propagate = False

In [7]:
debug_requests_off()

In [8]:
url = 'http://192.168.1.5:5000'
volume = {'volume':5}
r = requests.post(url + '/get_status', json = {'name':'1cc_2'})
r.content

b'{"remaining_volume":null,"syringe_loaded":false}\n'

In [3]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False) 

In [5]:
jubilee.home_all()

In [4]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

## Load labware


In [5]:
samples = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 2)
samples.manual_offset([(19.8,178.1),(132.5,177.6),(132.5,106.6)])

New manual offset applied to septavialrev1_44_holder_2000ul


In [6]:
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks.manual_offset([(31.3,268.1),(117.6,267),(117.9,210.9)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [7]:
tiprack = jubilee.load_labware('opentrons_96_tiprack_300ul.json', 0)
tiprack.manual_offset([(30.1, 77.0), (128.4, 77.4), (128.9, 14.9)])

New manual offset applied to opentrons_96_tiprack_300ul


In [8]:
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

## Load Tools

In [9]:
P300 = Pipette.Pipette.from_config(0, 'Pipette', 'P300_config.json')
jubilee.load_tool(P300)
P300.add_tiprack(tiprack)
P300.trash = trash[0]

In [10]:
syringe_10 = syringe.HTTPSyringe.from_config(1, "../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(2, "../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../science-jubilee/src/science_jubilee/tools/configs/1cc_2_syringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../science-jubilee/src/science_jubilee/tools/configs/1cc_3_syringe.json")

In [11]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [12]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3

In [14]:
#water_syringe.set_pulsewidth(1350, s = 10)

## Load water syringe with water

In [13]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [14]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [15]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 10)

In [17]:
water_syringe.set_pulsewidth(water_syringe.full_position+150, s = 500)

In [18]:
water_syringe.load_syringe(820, water_syringe.full_position+150)

Loaded syringe, remaining volume 820 uL


## Load ammonia syringe

In [17]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [19]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [20]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [21]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 2000)

In [23]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+250, s = 500)

In [24]:
ammonia_syringe.load_syringe(550, ammonia_syringe.full_position+250)

Loaded syringe, remaining volume 550 uL


## Load TEOS syringe

In [25]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [26]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [27]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 200)

In [29]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+200, s = 200)

In [30]:
teos_syringe.load_syringe(700, teos_syringe.full_position+200)

Loaded syringe, remaining volume 700 uL


In [31]:
mix_syringe.load_syringe(0, 1829)

Loaded syringe, remaining volume 0 uL


In [32]:
mix_syringe.set_pulsewidth(1829)

## Experiment

##### EtOH: 1851.8 ul
##### NH3OH: 67.6 ul
##### H20: 36 ul
##### TEOS: 44.6 ul

In [33]:
total_volume = 1851.8 + 67.6 + 36 + 44.6

In [34]:
etoh_frac = 1851.8 / total_volume
ammonia_frac = 67.6 / total_volume
water_frac = 36 / total_volume
teos_frac = 44.6/total_volume

In [35]:
target_vol = 1800

etoh_vol = target_vol*etoh_frac
ammonia_vol = target_vol*ammonia_frac
water_vol = target_vol*water_frac
teos_vol = target_vol*teos_frac

In [36]:
#etoh transfer strategy
max_trans_volume = 290

n_full_transfers = int(np.floor(etoh_vol / max_trans_volume))
final_etoh_vol = etoh_vol - n_full_transfers*max_trans_volume

In [37]:
print('etoh vol: ', etoh_vol)
print('sum of transfers: ', n_full_transfers*max_trans_volume + final_etoh_vol)

etoh vol:  1666.6200000000001
sum of transfers:  1666.6200000000001


In [38]:
wash_location_1 = stocks[1]
wash_location_2 = stocks[2]

mix_vol = 2000
max_trans_volume = 290

In [59]:
def prepare_silica_precursors(sample_well, etoh_vol, water_vol, ammonia_vol):
    """
    Mix precursors (etoh, water, ammonia) at sample_well)
    """

    # transfer ethanol

    jubilee.pickup_tool(P300)
    P300.pickup_tip()

    #etoh transfer strategy ####
    

    n_full_transfers = int(np.floor(etoh_vol / max_trans_volume))
    final_etoh_vol = etoh_vol - n_full_transfers*max_trans_volume
    #########

    # etoh transfer execution
    for k in range(n_full_transfers):
        P300.aspirate(max_trans_volume, stocks[0])
        P300.dispense(max_trans_volume, sample_well.bottom(+22), s = 1000)
        time.sleep(3)
    # final transfer to hit vol
    P300.aspirate(final_etoh_vol, stocks[0])
    P300.dispense(final_etoh_vol, sample_well.bottom(+22))
    time.sleep(3)

    P300.drop_tip()
    jubilee.park_tool()


    # Transfer Ammonia
    jubilee.pickup_tool(ammonia_syringe)

    ammonia_syringe.dispense(ammonia_vol, sample_well.bottom(+20), s = 10)
    time.sleep(3)

    jubilee.park_tool()

    # Transfer water
    jubilee.pickup_tool(water_syringe)

    water_syringe.dispense(water_vol, sample_well.bottom(+20), s = 10)
    time.sleep(3)

    jubilee.park_tool()


    # mix 
    jubilee.pickup_tool(mix_syringe)
    mix_loc = samples[i].bottom(+5)
    print('mixing')
    mix_syringe.mix(mix_vol, 5, sample_well.bottom(+5), t_hold = 3, s_aspirate = 2000, s_dispense = 500)

    # wash syringe 
    mix_syringe.mix(mix_vol, 3, wash_location_1.bottom(+5), t_hold = 3, s_aspirate = 2000, s_dispense = 500)
    mix_syringe.mix(mix_vol, 3, wash_location_2.bottom(+5), t_hold = 3, s_aspirate = 2000, s_dispense = 500)

    jubilee.park_tool()




In [55]:
def add_teos(sample_well, teos_vol):

    jubilee.pickup_tool(teos_syringe)

    teos_syringe.dispense(teos_vol, sample_well.bottom(+20))
    time.sleep(3)
                          
    jubilee.park_tool()
    jubilee.pickup_tool(mix_syringe)
    

    mix_syringe.mix(mix_vol, 5, sample_well.bottom(+5), t_hold = 3, s_aspirate = 2000, s_dispense = 500)

    mix_syringe.mix(mix_vol, 3, wash_location_1, t_hold = 3, s_aspirate = 2000, s_dispense = 500)
    mix_syringe.mix(mix_vol, 3, wash_location_2, t_hold = 3, s_aspirate = 2000, s_dispense = 500)

    jubilee.park_tool()
    

In [55]:
jubilee.pickup_tool(mix_syringe)

In [60]:
mix_syringe.mix(2000, 5, samples[1].bottom(+5), t_hold = 3, s_aspirate = 2000, s_dispense = 500)

aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL


In [43]:
P300.drop_tip()

In [61]:
jubilee.park_tool()

In [62]:
add_teos(sample_location, teos_vol)

aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL


In [63]:
n_samples = 5

for i in range(2, n_samples):
    sample_location = samples[i]
    prepare_silica_precursors(sample_location, etoh_vol, water_vol, ammonia_vol)
    add_teos(sample_location, teos_vol)

mixing
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
mixing
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 uL
aspirating 2000 uL
dispensing 2000 u

In [61]:
jubilee.park_tool()

send: b'POST /machine/code HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.32.3\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 4\r\n\r\n'
send: b'M114'
reply: 'HTTP/1.1 500 only rr_upload is supported for POST requests\r\n'
header: Connection: close
send: b'GET /rr_model?key=seqs HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.32.3\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Cache-Control: no-cache, no-store, must-revalidate
header: Pragma: no-cache
header: Expires: 0
header: Content-Type: application/json
header: Content-Length: 245
header: Connection: close
send: b'GET /rr_gcode?gcode=M114 HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.32.3\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Cache-Control: no-cache, no-store, must-revali